In [2]:
import ray, random, os 
ray.shutdown()

In [3]:
import ray, random, os 
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Users/floriankockler/raytemp/')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from multi_agent import MultiAgent
from manager import Manager
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.rllib.utils import check_env
from ray.tune.logger import pretty_print

2023-09-29 18:25:45,152	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-09-29 18:26:03,101	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [4]:
from ray.rllib.utils import check_env
check_env(MultiAgent())

controller: agents:  {'ABT.US': <worker_standlone.WorkerStandAlone object at 0x1061cc160>, 'AMGN.US': <worker_standlone.WorkerStandAlone object at 0x10b366f50>, 'BDX.US': <worker_standlone.WorkerStandAlone object at 0x10b367a00>, 'BMY.US': <worker_standlone.WorkerStandAlone object at 0x10b366ec0>, 'HUM.US': <worker_standlone.WorkerStandAlone object at 0x32987aa10>, 'JNJ.US': <worker_standlone.WorkerStandAlone object at 0x10b367100>, 'LLY.US': <worker_standlone.WorkerStandAlone object at 0x32987ba30>, 'MDT.US': <worker_standlone.WorkerStandAlone object at 0x32987b9a0>, 'MRK.US': <worker_standlone.WorkerStandAlone object at 0x32a6ca7d0>, 'PFE.US': <worker_standlone.WorkerStandAlone object at 0x32987baf0>, 'SYK.US': <worker_standlone.WorkerStandAlone object at 0x32a6cb7f0>, 'TMO.US': <worker_standlone.WorkerStandAlone object at 0x32a6cb760>, 'UNH.US': <worker_standlone.WorkerStandAlone object at 0x32b2d6590>}


In [6]:
multi_agent_env = MultiAgent()

# Initialize environment
obs, info = multi_agent_env.reset()
# print("Initial Observations:", obs)

# Take exactly 10 steps
for step in range(10):
    actions = {}
    
    # Collect sample actions for each agent
    for agent_id, agent_obs in obs.items():
        action_space = multi_agent_env.action_space[agent_id]
        action = action_space.sample()
        actions[agent_id] = action
    
    # Step the environment
    obs, reward, done, _, info = multi_agent_env.step(actions)
    
    # print(f"Step: {step + 1}")
    # print(f"Actions: {actions}")
    # print(f"Observations: {obs}")
    # print(f"Rewards: {reward}")
    # print(f"Done flags: {done}")
    # print(f"Info: {info}")

controller: agents:  {'ABT.US': <worker_standlone.WorkerStandAlone object at 0x32a6cbb80>, 'AMGN.US': <worker_standlone.WorkerStandAlone object at 0x32a6ca140>, 'BDX.US': <worker_standlone.WorkerStandAlone object at 0x32a6cbd60>, 'BMY.US': <worker_standlone.WorkerStandAlone object at 0x32a6c80a0>, 'HUM.US': <worker_standlone.WorkerStandAlone object at 0x32a6c9f00>, 'JNJ.US': <worker_standlone.WorkerStandAlone object at 0x32b366620>, 'LLY.US': <worker_standlone.WorkerStandAlone object at 0x32a6cbf40>, 'MDT.US': <worker_standlone.WorkerStandAlone object at 0x32b367640>, 'MRK.US': <worker_standlone.WorkerStandAlone object at 0x32b3675b0>, 'PFE.US': <worker_standlone.WorkerStandAlone object at 0x32c4870a0>, 'SYK.US': <worker_standlone.WorkerStandAlone object at 0x32b367700>, 'TMO.US': <worker_standlone.WorkerStandAlone object at 0x32c487430>, 'UNH.US': <worker_standlone.WorkerStandAlone object at 0x32c4873a0>}
controller: agents:  {'ABT.US': <worker_standlone.WorkerStandAlone object at 0x3

(raylet) [2023-10-01 02:05:37,695 E 2937 44933] (raylet) file_system_monitor.cc:111: /Users/floriankockler/raytemp/session_2023-09-29_18-25-40_956736_2059 is over 95% full, available space: 12251762688; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-10-01 02:05:47,775 E 2937 44933] (raylet) file_system_monitor.cc:111: /Users/floriankockler/raytemp/session_2023-09-29_18-25-40_956736_2059 is over 95% full, available space: 12164476928; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-10-01 02:05:57,856 E 2937 44933] (raylet) file_system_monitor.cc:111: /Users/floriankockler/raytemp/session_2023-09-29_18-25-40_956736_2059 is over 95% full, available space: 12212039680; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-10-01 02:06:07,937 E 2937 44933] (raylet) file_system_monitor.cc:111: /Users/floriankockler/raytemp/session_2023-09-29_18-25-40_956736_2059 is over 95%

In [4]:
multi_agent_env = MultiAgent()


for episode in range(1):
    obs, info = multi_agent_env.reset()
    done = {"__all__": False}
    
    print(f"Episode {episode + 1}")

    while not done["__all__"]:
        actions = {}
        
        # Collect actions for each agent
        for agent_id, agent_obs in obs.items():
            action_space = multi_agent_env.action_space[agent_id]
            action = action_space.sample()
            actions[agent_id] = action

        # Step the environment
        obs, reward, done, _, info = multi_agent_env.step(actions)

        # print(f"Actions: {actions}")
        # print(f"Observations: {obs}")
        # print(f"Rewards: {reward}")
        # print(f"Done flags: {done}")
        # print(f"Info: {info}")

    print("Episode done!")

Episode 1
Controller day:  1
Controller day:  2
Controller day:  3
Controller day:  4
Controller day:  5
Controller day:  6
Controller day:  7
Controller day:  8
Controller day:  9
Controller day:  10
Controller day:  11
Controller day:  12
Controller day:  13
Controller day:  14
Controller day:  15
Controller day:  16
Controller day:  17
Controller day:  18
Controller day:  19
Controller day:  20
Controller day:  21
Controller day:  22
Controller day:  23
Controller day:  24
Controller day:  25
Controller day:  26
Controller day:  27
Controller day:  28
Controller day:  29
Controller day:  30
Controller day:  31
Controller day:  32
Controller day:  33
Controller day:  34
Controller day:  35
Controller day:  36
Controller day:  37
Controller day:  38
Controller day:  39
Controller day:  40
Controller day:  41
Controller day:  42
Controller day:  43
Controller day:  44
Controller day:  45
Controller day:  46
Controller day:  47
Controller day:  48
Controller day:  49
Controller day:  50

Controller day:  4801
Controller day:  4802
Controller day:  4803
Controller day:  4804
Controller day:  4805
Controller day:  4806
Controller day:  4807
Controller day:  4808
Controller day:  4809
Controller day:  4810
Controller day:  4811
Controller day:  4812
Controller day:  4813
Controller day:  4814
Controller day:  4815
Controller day:  4816
Controller day:  4817
Controller day:  4818
Controller day:  4819
Controller day:  4820
Controller day:  4821
Controller day:  4822
Controller day:  4823
Controller day:  4824
Controller day:  4825
Controller day:  4826
Controller day:  4827
Controller day:  4828
Controller day:  4829
Controller day:  4830
Controller day:  4831
Controller day:  4832
Controller day:  4833
Controller day:  4834
Controller day:  4835
Controller day:  4836
Controller day:  4837
Controller day:  4838
Controller day:  4839
Controller day:  4840
Controller day:  4841
Controller day:  4842
Controller day:  4843
Controller day:  4844
Controller day:  4845
Controller

In [ ]:
if os.name == 'nt':
    path_to_save = "C:\\GitHub\\ray\\workertune"
else:
    path_to_save = "/Volumes/SSD980/ray/multiagent2"


env = MultiAgent()
def env_creator(env_config):
    return MultiAgent()  

register_env("MultiAgent", env_creator)

asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=2000,
    grace_period=200,
    reduction_factor=3,
    brackets=1,
)

def create_policy_spec(agent_id):
    # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[agent_id],
        action_space=env.action_space[agent_id],
        config={}
    )

controller_policy_spec = PolicySpec(
    observation_space=env.observation_space['controller'],
    action_space=env.action_space['controller'],
    config={}
)

policies = {
    "controller_policy": controller_policy_spec,
}

for agent_id in env.agents:
    policies[agent_id] = create_policy_spec(agent_id)

def policy_mapping_fn(agent_id, episode=None, agent=None, **kwargs):
    if agent_id == 'controller':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "controller_policy"
    elif agent_id in env.agents:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "MultiAgent",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 3, 
        #Change for Debugging
        "log_level": "ERROR",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean", 
    num_samples=10,
    resume=False,
    mode="max",
    config=param_space, 
    local_dir=path_to_save,
    search_alg=None,
    scheduler=asha_scheduler,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=120),
    max_concurrent_trials=2,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

print(pretty_print(analysis.last_result))
print("Best hyperparameters found were: ", pretty_print(analysis.best_config))

In [ ]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)



analysis = tune.run(
        "A2C",
        metric="episode_reward_mean",
        mode="max",
        config={
            "env": "worker",
            "env_config": {"initial_capital": 1e6},
            "lr": tune.uniform(1e-5, 1e-4),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
}, )

In [ ]:


n_iterations = 30000


env = WorkerStandAlone(data=data)

state = env.reset()

for _ in range(n_iterations):
    action = env.action_space.sample()  # Generate random actions for the manager
    worker_dones = {...}  # You need to provide the values for worker_dones
    worker_truncateds = {...}  # You need to provide the values for worker_truncateds

    obs, reward, done, truncated, info = env.step(action)

    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()